In [1]:
import os
import sys
import numpy as np
import xarray as xr

os.chdir('/h/u145/liuxinrui/CROP/')
sys.path.insert(0, '/h/u145/liuxinrui/CROP/')

In [2]:
## load parameters
from core_fct.fct_loadP_CROP import load_ISIMIP_param
Par = load_ISIMIP_param()

loading primary parameters
>>>  Running split_region  for Southern Asia <<<
>>>  Running split_region  for South-eastern Asia <<<
>>>  Running split_region  for Africa <<<
>>>  Running split_region  for Northern America <<<
>>>  Running split_region  for USA <<<
>>>  Running split_region  for CAN <<<
>>>  Running split_region  for Latin America and the Caribbean <<<
>>>  Running split_region  for BRA <<<
>>>  Running split_region  for Eastern Asia <<<
>>>  Running split_region  for CHN <<<
>>>  Running split_region  for Central Asia <<<
>>>  Running split_region  for Western Asia <<<
>>>  Running split_region  for Europe <<<
>>>  Running split_region  for RUS <<<
>>>  Running split_region  for Oceania <<<
>>>  Running split_region  for AUS <<<


In [3]:
print(Par['a_CO2'])

<xarray.DataArray 'a_CO2' (spc_crop: 6, reg_land: 292, irr: 2, mod_YD_crop: 8)>
array([[[[4.73160929e-01, 3.49283470e-03, 6.21746448e-02, ...,
          1.55579522e-01, 5.38869015e-01, 3.02997567e-03],
         [6.27437923e-01, 1.23066396e-02, 1.72938793e-01, ...,
          1.22508444e-01, 1.68584537e+00, 7.79334220e-02]],

        [[           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan]],

        [[4.95577160e-01, 8.23109217e-04, 1.51915659e-02, ...,
          1.47573277e-01, 8.83099220e-01, 8.79210479e-02],
         [5.03885655e-01, 9.34965206e-03, 4.05164461e-02, ...,
          5.35729158e-02, 1.15805030e+00, 6.54001553e-02]],

        ...,

        [[4.49948543e-01, 2.81581285e-03, 3.92251675e-02, ...,
          1.04572163e-01, 6.36116089e-01, 6.39276013e-03],
         [3.12283737e-01, 8.04190135e-03, 1.55

In [4]:
## load drivers
from core_fct.fct_loadD_CROP import load_Nfertl_scen, load_Ndep_scen
For_Nfertl = load_Nfertl_scen(datasets=['ISIMIP3b'])
print(For_Nfertl)

For_Ndep = load_Ndep_scen(datasets=['ISIMIP3b'])
print(For_Ndep)

<xarray.Dataset>
Dimensions:        (year: 165, spc_crop: 6, reg_land: 292, scen_N_fertl: 2)
Coordinates:
  * reg_land       (reg_land) int64 1 2 3 4 5 6 7 ... 287 288 289 290 291 292
  * scen_N_fertl   (scen_N_fertl) <U16 'ISIMIP3b-1850soc' 'ISIMIP3b-2015soc'
  * year           (year) int64 1850 1851 1852 1853 1854 ... 2011 2012 2013 2014
  * spc_crop       (spc_crop) <U3 'mai' 'ri1' 'ri2' 'soy' 'swh' 'wwh'
    reg_land_name  (reg_land) object 'Afghanistan' 'Aland' ... 'Zimbabwe'
    reg_land_code  (reg_land) object 'AFG' 'ALA' 'ALB' ... 'YEM' 'ZMB' 'ZWE'
Data variables:
    N_fertl        (year, spc_crop, reg_land, scen_N_fertl) float64 0.0 ... 3...
<xarray.Dataset>
Dimensions:        (year: 251, reg_land: 292, scen_N_dep: 5)
Coordinates:
  * reg_land       (reg_land) int64 1 2 3 4 5 6 7 ... 287 288 289 290 291 292
  * scen_N_dep     (scen_N_dep) <U18 'ISIMIP3b-1850soc' ... 'ISIMIP3b-ssp585soc'
    reg_land_name  (reg_land) object 'Afghanistan' 'Aland' ... 'Zimbabwe'
    reg_land_cod

In [5]:
## generate MC parameters
from core_fct.fct_genMC import generate_config
Par = generate_config(Par, 50)

generating MC configurations


In [6]:
## load necessary input variables
For_clim = xr.load_dataset('./input_data/observations/crop/land-climate_ISIMIP3a.nc')
For_co2 = xr.load_dataset('./input_data/observations/crop/concentrations_ISIMIP3b.nc').sel(scen='historical', drop=True)

## set run options
ind0, ind1 = 1901, 2014
For = xr.merge([For_clim, For_co2, For_Nfertl, For_Ndep]).sel(year=slice(ind0, ind1))

## format drivers
## Initial CO2 here is 399.95 ppm
For['D_CO2'] = For.CO2 - Par['CO2_0']
For['D_Tl'] = For.Tl - For.Tl.sel(year=slice(1901, 1930)).mean('year')
For['D_Pl'] = For.Pl - For.Pl.sel(year=slice(1901, 1930)).mean('year')
For = For.drop(['CO2', 'Tl', 'Pl'])
For = For.fillna(0).sel(year=slice(1900)).combine_first(For)

# generate MC drivers
# from core_fct.fct_genMC import generate_drivers
# For = generate_drivers(For, 500)

In [7]:
## import offline crop emulator
from core_fct.mod_process_CROP import CROP
CROP = CROP()

In [ ]:
Out = CROP(None, Par, For, var_keep=['D_Tgs', 'D_Pgs', 'NI', 'RC', 'RT', 'RP', 'RN', 'YD'])
Out.to_netcdf('./results/test.nc')

OSCAR_v3_CROP running
year = 2014 (nt = 2)
total running time: 2.5 minutes


PermissionError: [Errno 13] Permission denied: b'/h/u145/liuxinrui/CROP/results/test.nc'

In [ ]:
import numpy as np
import xarray as xr

maskfile = './input_data/mask/mask_0.5x0.5.nc'
ds = xr.load_dataset('./results/test.nc')

from core_fct.fct_pre_CROP import create_global_map
import matplotlib.pyplot as plt

ax = create_global_map(ds['YD'].sel(spc_crop='mai', irr='firr', mod_YD_crop='CYGMA1p74', mod_RN_fct='MM', data_N_fertl='ISIMIP3a', data_N_dep='ISIMIP3a', mod_bnf_soy='Ma_2022', year=slice(1981, 2010)).mean(dim=['year', 'mod_clim_gcm']), maskfile, levels=np.linspace(0,10,11))
ax.set_title('Maize (tDM/ha)', fontsize=14)
plt.tight_layout()
plt.savefig('mai.png')

In [3]:
print(CROP.proc_levels())
print(CROP.var_in)
print(CROP.var_mid)
print(CROP.var_out)
print(CROP._processes.keys())
print(CROP.var_prog)
print(CROP.var_node)
print(CROP.var_diag)

{0: [], 1: ['D_Pgs', 'RC', 'D_Tgs', 'NI'], 2: ['RN', 'D_WI', 'RT'], 3: ['RW'], 4: ['YD']}
{'N_dep', 'N_fertl', 'D_CO2', 'D_Pl', 'D_Tl'}
{'RN', 'D_Pgs', 'RC', 'D_Tgs', 'RW', 'D_WI', 'NI', 'RT'}
{'YD'}
dict_keys(['RC', 'D_Tgs', 'RT', 'D_Pgs', 'D_WI', 'RW', 'NI', 'RN', 'YD'])
set()
set()
{'RN', 'D_Pgs', 'RC', 'D_Tgs', 'RW', 'D_WI', 'NI', 'RT', 'YD'}


In [31]:
func1 = lambda x, a: a*x + 1
func2 = lambda x, a: np.log1p(a*x) + 1
func3 = lambda x, a, b: (1 + a*np.log1p(x)) / (1 + b*x)
func4 = lambda x, a, b: -b*np.expm1(-a*x) + 1
func5 = lambda x, a, b: np.exp(-a*(x-b)**2+a*b**2)
func6 = lambda x, a, b: np.log1p(np.exp(-a*(x-b)**2 + a*b**2)) / np.log(2)
func7 = lambda x, a, b: (np.exp(-a*b)+np.exp(a*b))/(np.exp(a*(x-b))+np.exp(-a*(x-b)))
func8 = lambda x, a, b: (np.exp(a*b) + 1) / (np.exp(-a*(x-b)) + 1)
func9 = lambda x, a, b, c: (np.exp(-a*b)+np.exp(c*b))/(np.exp(a*(x-b))+np.exp(-c*(x-b)))

RC_switch = Par['RC_switch'].loc[{'mod_YD_crop':'CYGMA1p74'}]
D_CO2 = For['D_CO2']
a_CO2 = Par['a_CO2'].loc[{'spc_crop':'mai', 'reg_land':24, 'mod_YD_crop':'CYGMA1p74'}]
b_CO2 = Par['b_CO2'].loc[{'spc_crop':'mai', 'reg_land':24, 'mod_YD_crop':'CYGMA1p74'}]

RC_1st = func1(D_CO2/Par.CO2_pi, a_CO2)
RC_2nd = func2(D_CO2/Par.CO2_pi, a_CO2)
RC_3rd = func3(D_CO2/Par.CO2_pi, a_CO2, b_CO2)
RC_4th = func4(D_CO2/Par.CO2_pi, a_CO2, b_CO2)
RC_5th = func5(D_CO2/Par.CO2_pi, a_CO2, b_CO2)
RC_6th = func6(D_CO2/Par.CO2_pi, a_CO2, b_CO2)
RC = xr.concat([RC_1st.assign_coords(fct_YD=1), RC_2nd.assign_coords(fct_YD=2), RC_3rd.assign_coords(fct_YD=3), RC_4th.assign_coords(fct_YD=4), RC_5th.assign_coords(fct_YD=5), RC_6th.assign_coords(fct_YD=6)], dim='fct_YD')
RC0 = (RC_switch*RC).sum('fct_YD', min_count=1)

print(RC)
print(RC_switch)
print(RC0)

a_WI = Par['a_WI'].loc[{'spc_crop':'mai', 'reg_land':368, 'mod_YD_crop':'SIMPLACE-LINTUL5'}]
b_WI = Par['b_WI'].loc[{'spc_crop':'mai', 'reg_land':368, 'mod_YD_crop':'SIMPLACE-LINTUL5'}]
Pgs_0 = Par['Pgs_0'].loc[{'spc_crop':'mai', 'reg_land':368, 'mod_YD_crop':'SIMPLACE-LINTUL5'}]

D_WI = For['D_Pl'].loc[{'reg_land':368}]*Par['w_Pgs'].loc[{'spc_crop':'mai', 'reg_land':368}]/Pgs_0

RW = func8(D_WI, a_WI, b_WI)
print(RW.max(), RW.min())

<xarray.DataArray ()>
array(2.49152668)
Coordinates:
    spc_crop     <U3 'mai'
    reg_land     int64 368
    mod_YD_crop  <U16 'SIMPLACE-LINTUL5' <xarray.DataArray ()>
array(0.41708516)
Coordinates:
    spc_crop     <U3 'mai'
    reg_land     int64 368
    mod_YD_crop  <U16 'SIMPLACE-LINTUL5'


In [19]:
for n in np.arange(50):
    print('(', n, ')', Par['RC_switch'].loc[{'config':n}].values.tolist().index(1)+1, Par['RT_switch'].loc[{'config':n}].values.tolist().index(1)+1, Par['RW_switch'].loc[{'config':n}].values.tolist().index(1)+1)

=== 0 ===
( 0 ) 4 7 9
=== 1 ===
( 1 ) 2 7 8
=== 2 ===
( 2 ) 2 7 8
=== 3 ===
( 3 ) 6 9 8
=== 4 ===
( 4 ) 1 9 9
=== 5 ===
( 5 ) 4 7 8
=== 6 ===
( 6 ) 3 9 8
=== 7 ===
( 7 ) 2 5 8
=== 8 ===
( 8 ) 6 9 8
=== 9 ===
( 9 ) 1 9 9
=== 10 ===
( 10 ) 4 7 8
=== 11 ===
( 11 ) 3 7 9
=== 12 ===
( 12 ) 6 9 8
=== 13 ===
( 13 ) 1 9 9
=== 14 ===
( 14 ) 3 7 9
=== 15 ===
( 15 ) 2 7 8
=== 16 ===
( 16 ) 4 7 8
=== 17 ===
( 17 ) 4 7 8
=== 18 ===
( 18 ) 2 7 8
=== 19 ===
( 19 ) 3 7 9
=== 20 ===
( 20 ) 3 9 8
=== 21 ===
( 21 ) 3 9 8
=== 22 ===
( 22 ) 4 7 8
=== 23 ===
( 23 ) 4 7 9
=== 24 ===
( 24 ) 4 7 9
=== 25 ===
( 25 ) 6 9 8
=== 26 ===
( 26 ) 4 7 8
=== 27 ===
( 27 ) 3 7 9
=== 28 ===
( 28 ) 3 7 9
=== 29 ===
( 29 ) 4 9 8
=== 30 ===
( 30 ) 4 7 9
=== 31 ===
( 31 ) 4 7 9
=== 32 ===
( 32 ) 3 9 8
=== 33 ===
( 33 ) 2 5 8
=== 34 ===
( 34 ) 6 9 8
=== 35 ===
( 35 ) 3 7 9
=== 36 ===
( 36 ) 2 5 8
=== 37 ===
( 37 ) 6 9 8
=== 38 ===
( 38 ) 1 9 9
=== 39 ===
( 39 ) 2 7 8
=== 40 ===
( 40 ) 4 9 8
=== 41 ===
( 41 ) 3 7 9
=== 42 ===
(

In [2]:
Out = CROP(None, Par, For, var_keep=['D_Tgs', 'D_Pgs', 'NI', 'RC', 'RT', 'RW', 'RN', 'YD'])
Out.to_netcdf('./results/test.nc')

{1: ['D_Tgs', 'NI', 'RC', 'D_Pgs'], 2: ['RN', 'D_WI', 'RT'], 3: ['RW'], 4: ['YD']}
OSCAR_v3_CROP running
year = 2015 (nt = 3)
total running time: 1.9 minutes


In [4]:
print(Out['RC'].max(), Out['RC'].min())
print(Out['RT'].max(), Out['RT'].min())
print(Out['RW'].max(), Out['RW'].min())
print(Out['RN'].max(), Out['RN'].min())
print(Out['YD'].max(), Out['YD'].min())

<xarray.DataArray 'RC' ()>
array(327.134545) <xarray.DataArray 'RC' ()>
array(0.00252701)
<xarray.DataArray 'RT' ()>
array(inf) <xarray.DataArray 'RT' ()>
array(0.)
<xarray.DataArray 'RW' ()>
array(6911035.22484634) <xarray.DataArray 'RW' ()>
array(2.12064667e-08)
<xarray.DataArray 'RN' ()>
array(128.41903588) <xarray.DataArray 'RN' ()>
array(-164.38372841)
<xarray.DataArray 'YD' ()>
array(inf) <xarray.DataArray 'YD' ()>
array(-inf)


In [7]:
year = Out.year.rolling(year=5, center=True).mean()
year, D_Tgs = xr.broadcast(year, Out['D_Tgs'])
print(year.loc[{'year':2000}])

<xarray.DataArray 'year' (spc_crop: 6, reg_land: 222, mod_clim_gcm: 4)>
array([[[2000., 2000., 2000., 2000.],
        [2000., 2000., 2000., 2000.],
        [2000., 2000., 2000., 2000.],
        ...,
        [2000., 2000., 2000., 2000.],
        [2000., 2000., 2000., 2000.],
        [2000., 2000., 2000., 2000.]],

       [[2000., 2000., 2000., 2000.],
        [2000., 2000., 2000., 2000.],
        [2000., 2000., 2000., 2000.],
        ...,
        [2000., 2000., 2000., 2000.],
        [2000., 2000., 2000., 2000.],
        [2000., 2000., 2000., 2000.]],

       [[2000., 2000., 2000., 2000.],
        [2000., 2000., 2000., 2000.],
        [2000., 2000., 2000., 2000.],
        ...,
...
        ...,
        [2000., 2000., 2000., 2000.],
        [2000., 2000., 2000., 2000.],
        [2000., 2000., 2000., 2000.]],

       [[2000., 2000., 2000., 2000.],
        [2000., 2000., 2000., 2000.],
        [2000., 2000., 2000., 2000.],
        ...,
        [2000., 2000., 2000., 2000.],
        [2000., 2